In [1]:
'''
SCRIPT PER PROVARE configurazioni SVM
'''

import time
t_orig = time.process_time()
last_time = 0

import warnings
warnings.simplefilter("ignore")

import codecs
import csv
import argparse
import os
import numpy as np
import pickle

from time import time
from dill import dill
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import cross_validation, svm
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# modulo per confrontare il risultato delle regex
# richiede entrambi i db già etichettati
import tortellino_parmigiano 
import crea_db


import evaluation_report # stampa confusion_matrix e altro
from text_tokenizer import get_tokenizer 
from nltk.stem.snowball import ItalianStemmer

/home/utente_locale/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
ita_stopwords = set(stopwords.words('italian'))
ita_stemmer = ItalianStemmer()
ita_stemming = ita_stemmer.stem
def get_tokenizer1(doc): #(word_ngrams=None, char_ngrams=None, stopwords=None, additional_extractors=None):
    # rimuove le stopwords
    doc = filter(lambda x: x not in ita_stopwords, word_tokenize(doc))
    # stemma le parole
    doc = [ita_stemmer.stem(a) for a in doc]
    return doc

def get_tokenizer(doc): #(word_ngrams=None, char_ngrams=None, stopwords=None, additional_extractors=None):
    doc = filter(lambda x: x not in ita_stopwords, word_tokenize(doc))
    #doc = map(ita_stemming, doc)
    #doc = [ita_stemmer.stem(a) for a in doc]
    temp = ''
    for par in doc:
        temp = '{} {}'.format(temp, ita_stemming(par))
    return temp

In [3]:
class CestinoTraining:
    def __init__(self, features, labels, categories):
        self.stemmed = list(map(get_tokenizer, texts))
        self.features = features
        self.labels = np.array(labels)
        self.categories = categories

In [5]:
# list of possible algorithms
algos = ['svm-lin', 'svm-rbf', 'nb', 'decisiontree', 'randomforest', 'regex', 'svm-multi']
    # svm-rbf might not apply to text problems: https://charlesmartin14.wordpress.com/2012/02/06/kernels_part_1/

DATA_FOLDER = "/home/backup/dati_e_csv/" # DATI
# ci assicuriamo siano presenti i DB
training_db_name = crea_db.build_training(OUTPUT=DATA_FOLDER)
test_db_name = crea_db.build_test(OUTPUT=DATA_FOLDER) ################################

    
if os.path.exists('/home/backup/dati_e_csv/training_stemmed-bubu.pickle'):
    print('Trovato salvataggio training set')
    with open('/home/backup/dati_e_csv/training_stemmed-bubu.pickle', 'rb') as file:
        salvataggio = pickle.load(file)
    texts = salvataggio.features
    texts_stemmed = salvataggio.stemmed
    Y = salvataggio.labels
    print('Salvataggio caricato')
else:
    print('Creo il training set')
    
    texts = list()
    labels = list()
    names= []

    # load training dataset # possiamo usare sqlite
    with codecs.open(os.path.join(DATA_FOLDER, training_db_name), 'r', errors='ignore') as file:
        reader = csv.reader(file, delimiter=';')
        etichette = next(reader)
        for row in reader:
            # load documents
            #print('Nome {}, etichette {}, conversazione{}'.format(row[10], row[:10], row[11]+row[12]))
            #raise SystemExit
            unito = (row[11]+row[12]).replace('#O:', '').replace('#C:', '') #concatena il testo cliente con operatore
            texts.append(unito)
            # load row name
            names.append(row[10])
            # load labels
            labels.append(list(map(lambda x: int(x), row[:10])))

    print('dimensioni:',len(texts), len(labels))
    # tokenization function # possiamo usare anche solo un canale cliente/operatore
    print('Inizio lo stemming')

    salvataggio = CestinoTraining(texts, labels, etichette)
    texts_stemmed = salvataggio.stemmed
    Y = salvataggio.labels
    with open('/home/backup/dati_e_csv/training_stemmed-bubu.pickle', 'wb') as file:
        pickle.dump(salvataggio, file, protocol=0)
    print('Training set creato e salvato')

Training sqlite e csv presenti!
Test sqlite e csv presenti!
Trovato salvataggio training set
Salvataggio caricato


In [5]:
#analyzer = get_tokenizer(stopwords=stopwords.words('italian'), #word_ngrams=[1],
#                         additional_extractors=None)



#algo = OneVsRestClassifier(SVC(kernel='linear', max_iter=5000), n_jobs=1)
algo = OneVsRestClassifier(LinearSVC(C=15), n_jobs=-1) #C=9
#print('Multilabel ', algo.multilabel_)

# pipeline definition
pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer=get_tokenizer1,# tokenizer=lambda x: x.split(' '),
                             max_df=0.6,min_df=0,max_features=1000)),#, max_df=0.5,min_df=0, max_features=1500)),# feature extraction,
        #analyzer=get_tokenizer1, 
    ('tfidf', TfidfTransformer(use_idf=True)),#use_idf=True)), # weighting
    #('chi2', SelectKBest(chi2, k=k)), # feature selection

########### Per il chi2 multilabel
# selected_features = [] 
# for label in labels:
#     selector = SelectKBest(chi2, k='all')
#     selector.fit(X, Y[label])
#     selected_features.append(list(selector.scores_))

# // MeanCS 
# selected_features = np.mean(selected_features, axis=0) > threshold
# // MaxCS
# selected_features = np.max(selected_features, axis=0) > threshold

    ('clf', algo) # learning algorithm
])

parameters = {#'vect__ngram_range': ((1, 1),(1, 2)),
              #'vect__max_df': (0.5,0.75,1.0),
              #'vect__min_df': (0,0.25,0.4),
              #'vect__max_features': (None,50,500,1000, 5000), # TO-DO: bisogna stemmare le features
              #'tfidf__use_idf': (True,False),
              #'tfidf__smooth_idf': (True,False)
              'clf__estimator__C': (15,18,20),
              #'clf__estimator__penalty': ('l1','l2')
             } # vuole min e max --> facciamo unigrammi e bigrammi



gs = GridSearchCV(pipeline, parameters, scoring='f1_macro', n_jobs=-1, verbose=10)


#print('Pipeline creata, avvio il modello')

print('Algoritmo pronto per essere fittato')

Algoritmo pronto per essere fittato


In [42]:
print(texts[1], len(texts))
print(texts_stemmed[1], len(texts_stemmed))

  sì  hanno hanno inviato documentazioni fonte signora  numero cliente vediamo signora  non ho il numero cliente  al  senza essere persone nella  il ministro con tutto toccata  perché allora nella documentazione che lei abbia inviato  risultava barche non vi era correttamente compilata  marcato alla Castelli fiscale su questa delicata  è molto  estremo non ce l' ho ancora signora questa anzi mi serve il venticinque di agosto  allora no signora lì di fronte a stare domani pomeriggio sul tasti perché i documenti lender lavorasse nel giro di quarantotto ore  quindi vedete ricontattare domani verso il tardi per vedere se sono pervenuti i documenti  va bene  io non ce l' ho ancora  non c'è ancora tutta  no con il nono bis signora non non non c'è proprio  non lo so signora io non lo vedo  grazie buongiorno  non c'è concorrenza  più o meno come fare la pratica perché è un po' di giorni ti aspetto  sì presa fatto tutto ha mandato il fax e avremo compilato un po' in  Como ottenendo Torino alle 

Grid Search celle larghe:

Best score: 0.464

Best parameters set:

clf__estimator__C: 1

tfidf__use_idf: True

vect__max_df: 0.5

vect__max_features: 1000

GRID SEARCH 2:

done in 822.022s
Best score: 0.488
Best parameters set:
	clf__estimator__C: 1.5
	vect__max_df: 0.5
	vect__max_features: 1500
    
GRID SEARCH 3:
done in 935.240s
Best score: 0.480
Best parameters set:
	clf__estimator__C: 10
	vect__max_features: 2500
	vect__min_df: 0
    
    
GS 4:
Best score: 0.488
Best parameters set:
	clf__estimator__C: 9
	vect__max_df: 0.5
	vect__max_features: 1500
	vect__min_df: 0


Best score: 0.471
Best parameters set:
	clf__estimator__C: 12
    
done in 1479.661s
Best score: 0.473
Best parameters set:
	clf__estimator__C: 18
    
done in 1254.545s
Best score: 0.481
Best parameters set:
	clf__estimator__C: 18

In [ ]:
print('grid search in action...')
t0 = time()
gs.fit(texts, Y)
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % gs.best_score_)
print("Best parameters set:")
best_parameters = gs.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [4]:
import pandas as pd
score_matrix = gs.cv_results_
df = pd.DataFrame(score_matrix)
df

AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [6]:
# controlla se esiste il modello per evitare di ricrearlo ogni volta
if False: #os.path.exists(os.path.join(DATA_FOLDER, 'OneVsRestClassifier')):
    with open(os.path.join(DATA_FOLDER, 'OneVsRestClassifier'), mode='rb') as modelfile:
        model = dill.load(modelfile)

else:
    model = pipeline.fit(texts, Y)
    print('Modello creato, verifico il test set')


    with open(os.path.join(DATA_FOLDER, 'OneVsRestClassifier'), mode='wb') as modelfile:
        dill.dump(model, modelfile)
        
print('Time to load/create the model: ', time.process_time()-last_time)
last_time = time.process_time()

KeyboardInterrupt: 

In [15]:
import time
print('Time to load create the model: ', time.process_time()-last_time)
last_time = time.process_time()

Time to load create the model:  0.022760550000043622


In [6]:
# importo il test set per la valutazione del modello
unito_test = []
names_test = []
test_texts = list()
test_labels = list()
#raise SystemExit

# load test data
# carica il test set
with codecs.open(os.path.join(DATA_FOLDER, test_db_name), 'r', errors='ignore') as file:
    reader = csv.reader(file, delimiter=';')
    etichette = next(reader)[3:]
    print(etichette)
    for row in reader:
        # load documents
        test_texts.append((row[1]+row[2]).replace('O:', '').replace('C:', ''))
        # load names
        names_test.append(row[0])
        # load labels
        test_labels.append(list(map(lambda x: int(x),row[-10:])))

['attivazione', 'canone rai', 'cessazione', 'contratto', 'credito', 'domiciliazione', 'fatturazione', 'gr-invito a chiamare - focus', 'non di competenza', 'richiamate']


In [7]:
Y_test = np.array(test_labels)
# apply model to data
predicted_labels = model.predict(test_texts) #np_array

# print evaluation report
evaluation_report.print_evaluation(Y_test, predicted_labels)
for i in range(predicted_labels.shape[1]):
    print(etichette[i], np.sum(predicted_labels[:,i]))
print('Decision function: ')

scores = model.decision_function(test_texts) #np_array

print('\n')
print('accuracy', accuracy_score(Y_test, predicted_labels))
print('precision micro',precision_score(Y_test, predicted_labels, average='micro') )
print('recall micro',recall_score(Y_test, predicted_labels, average='micro'))
print('f1 micro', f1_score(Y_test, predicted_labels, average='micro'))
print('precision macro', precision_score(Y_test, predicted_labels, average='macro') )
print('recall macro', recall_score(Y_test, predicted_labels, average='macro'))
print('f1 macro', f1_score(Y_test, predicted_labels, average='macro'))


NameError: name 'test_labels' is not defined

In [8]:
# score della cat credito 
cred_score = scores[:,4]
cred_pred = predicted_labels[:,4]
#print(len(cred_score), cred_pred)

positive = cred_score[cred_pred == 1]
neg = cred_score[cred_pred == 0]
#print(positive)
print('media score positivi predetti:   ', positive.mean())
print('media score negativi predetti:   ', neg.mean())
print('std score positivi predetti:     ', positive.std())
print('std score negativi predetti:     ', neg.std())

media score positivi predetti:    1.80927241817
media score negativi predetti:    -0.972798737
std score positivi predetti:      1.41654818067
std score negativi predetti:      0.720769286264


In [9]:
# confronto gli score dei doc annotati a mano con quelli assegnati dal modello
cred_score = scores[:,4]
cred_ammano = Y_test[:,4]
#print(len(cred_score), cred_ammano)

positive_ammano = cred_score[cred_ammano == 1]
neg_ammano = cred_score[cred_ammano == 0]
print(sum(cred_ammano), len(positive_ammano))
print('media score positivi:   ' , positive_ammano.mean())
print('media score negativi:   ', neg_ammano.mean())
print('std score positivi:     ', positive_ammano.std())
print('std score negativi:     ', neg_ammano.std())

156 156
media score positivi:    1.98479949207
media score negativi:    -0.129678476263
std score positivi:      1.72169001482
std score negativi:      1.31721210827


In [18]:
# confronto gli error rate per ciascuna classe
# predico il training set
Y_pred_train = model.predict(texts)

accuracy_test = []
accuracy_train = []
for i in range(predicted_labels.shape[1]):
    accuracy_test.append(accuracy_score(Y_test[:,i], predicted_labels[:,i]))
    accuracy_train.append(accuracy_score(Y[:,i], Y_pred_train[:,i]))
    

print ('error rate:')
print('{:>35} {:>18}'.format('test','train'))
for n in range(len(etichette[:10])):
      print('{:>23}{:>20}{:>10}'.format(etichette[n], round(1-accuracy_test[n],3), round(1-accuracy_train[n],3)))
    
print('''
    COME INTERPRETARE:
    
    Training error is low but is much lower than testing error - overfitting
    Both errors are low - ok
    Both errors are high - underfitting
    Training error is high but testing is low - error in implementation or very small dataset
''')

NameError: name 'model' is not defined

In [7]:
# ============================================    CHI2     ===================================================
# build training set
# stopwords, tf-idf

count_vect = CountVectorizer(texts, analyzer='word', stop_words=ita_stopwords)
X = count_vect.fit_transform(texts)

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X)
X_train_tf = tf_transformer.transform(X)
# dictionary for explore features selected k:index, v:feature name
inverse_vocab = { v:k for k,v in count_vect.vocabulary_.items() }


In [29]:
# statistical feature selection with chi2
# rank features according to the average or the maximum Chi-squared score across all labels
# led to most of the best classifiers while using less features. 
# fonte(http://ceur-ws.org/Vol-1094/bioasq2013_submission_8.pdf)
from sklearn.feature_selection import chi2, SelectKBest

def feature_selector(X,y):
    '''
    input:
    X: DT matrix
    y: labels vector
    n_features_perc: float: e.g. default 1.0, all features are retained
    
    output:
    X_reduced: DT matrix with only n_features
    '''
    n_features = 1500
    features_scores = [] 
    for cat in range(y.shape[1]):
        selector = SelectKBest(chi2, k='all')
        selector.fit(X, y[:,cat])
        features_scores.append(selector.scores_) # 10xn_feat matrix, score of each featrue for each class
    #print('selected_features:\n',len(features_scores), len(features_scores[0]))

    sorted_scores = sorted([item for sublist in features_scores for item in sublist], reverse = True)
    best_scores = sorted_scores[:n_features]
    #print(len(best_scores))
    
    # retrieve selected feature indices
    A = np.asarray(features_scores)
    selected_features = np.where(A > min(best_scores)) #retrieve indices of the best feature
    selected_features = selected_features[1]
    print(len(selected_features))
    #print('how many features we delete: \n', len(selected_features) - len(set(selected_features)), 'su',len(selected_features))

    # reduce DT matrix dimension
    selected_features = list(selected_features)
    X_reduced = X[:,selected_features] # cutting matrix features columns
    return X_reduced, selected_features


In [30]:
X_train_reduced, selected_feat = feature_selector(X_train_tf, Y)
X_train_reduced.shape


1499


(24379, 1499)

In [42]:
selected_feat[:10]

[161, 1196, 1221, 1313, 2570, 2974, 2997, 3164, 3494, 4089]

In [ ]:
# building the model
import time
model = None
start = time.process_time()    
model = OneVsRestClassifier(LinearSVC(C=10), n_jobs=1)
model.fit(X_train_reduced, Y)
last_time = time.process_time()
print('tempo per fittare il modello: ', last_time - start/60)


In [26]:
# build the test set to make predictions keeping attention for stemming
#count_vect = CountVectorizer(test_texts, analyzer='word', stop_words=ita_stopwords)
X_test = count_vect.transform(test_texts)
y_test = np.asarray(test_labels)

from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=True).fit(X_test)
X_test_tf = tf_transformer.transform(X_test)

X_test_reduced = X_test_tf[:,selected_feat]
print('reduced: ', X_test_reduced.shape, 'original: ', X_test_tf.shape)

reduced:  (424, 49) original:  (424, 90906)


In [27]:
# predictions
# apply model to data
predicted_labels = model.predict(X_test_reduced) #np_array

# print evaluation report
evaluation_report.print_evaluation(y_test, predicted_labels)
for i in range(predicted_labels.shape[1]):
    print(etichette[i], np.sum(predicted_labels[:,i]))
print('Decision function: ')

# scores = model.decision_function(test_texts) #np_array

# print('\n')
# print('accuracy', accuracy_score(Y_test, predicted_labels))
# print('precision micro',precision_score(Y_test, predicted_labels, average='micro') )
# print('recall micro',recall_score(Y_test, predicted_labels, average='micro'))
# print('f1 micro', f1_score(Y_test, predicted_labels, average='micro'))
# print('precision macro', precision_score(Y_test, predicted_labels, average='macro') )
# print('recall macro', recall_score(Y_test, predicted_labels, average='macro'))
# print('f1 macro', f1_score(Y_test, predicted_labels, average='macro'))

[[0 1 0 1 0 0 1 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [1 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]]
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
Confusion matrix attivazione
[[333  18]
 [ 59  14]]
Confusion matrix canone rai
[[359   2]
 [ 52  11]]
Confusion matrix cessazione
[[413   1]
 [  9   1]]
Confusion matrix contratto
[[329  21]
 [ 45  29]]
Confusion matrix credito
[[177  85]
 [ 29 133]]
Confusion matrix domiciliazione
[[398   5]
 [ 20   1]]
Confusion matrix fatturazione
[[275  25]
 [ 94  30]]
Confusion matrix gr-invito a chiamare - focus
[[260 141]
 [  8  15]]
Confusion matrix non di competenza
[[373   1]
 [ 48   2]]
Confusion matrix richiamate
[[417   4]
 [  3   0]]
                              precision    recall  f1-score   support

                 attivazione       0.44      0.19      0.27        73
                  canone rai       0.85      0.17      0.29        63
                  c